# Simplified ikeda check the parsed equations


In [7]:
import sympy as sp
import dill
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [8]:
with open('equations.dill', mode='rb') as file:
    equations = dill.load(file)

In [9]:
with open('symbols.dill', mode='rb') as file:
    symbols = dill.load(file)

## Proposed Formula of the Bilge Keel Roll Damping

In [10]:
equations.B_n_BK

Eq(B_n_BK, A_BK*exp*x_5*(B_B2K*x_3**B_B3K + B_BK))

In [11]:
equations.A_BK

Eq(A_BK, f_1*f_2*f_3)

In [12]:
equations.f_1

Eq(f_1, 0.3907*cdotx_1 - 2.5751*x_2 - 5.3769)

In [13]:
equations.f_2

Eq(f_2, 0.00255*x_6**2 + 0.122*x_6 + 0.4794)

In [14]:
equations.f_3

Eq(f_3, 4.3944*x_7**2 - 0.0733*x_7*x_8**2 - 0.01185*x_7 + 0.00189*x_8)

In [17]:
equations._asdict()

OrderedDict([('B_44', Eq(B_44, B_BK + B_E + B_F + B_L + B_W)),
             ('B_F', Eq(B_F, 4*c_f*omega*phi_a*r_f**3*rho*s_f/(3*pi))),
             ('c_f', Eq(c_f, 2.38301080148622*sqrt(phi_a**2*r_f**2/(T*v)))),
             ('r_f', Eq(r_f, (B*C_b + 0.2465*C_b*d - 2*G*O + 0.887)/pi)),
             ('s_f', Eq(s_f, B*C_b + 1.75*L_PP*d)),
             ('x_1', Eq(x_1, B/d)),
             ('x_2', Eq(x_2, C_b)),
             ('x_3', Eq(x_3, C_m)),
             ('x_4', Eq(x_4, G*O/d)),
             ('x_5', Eq(x_5, omega_n)),
             ('B_n_W',
              Eq(B_n_W, A_1*exp*(-A_2*log(x_5) - 0.694444444444444*A_3**2)/x_5)),
             ('A_1', Eq(A_1, AA_1*A_13 + A_11*x_4**2 + A_12*x_4)),
             ('A_2',
              Eq(A_2, -1.402*x_4**3 + 7.189*x_4**2 - 10.993*x_4 + 9.45)),
             ('A_3',
              Eq(A_3, AA_3 + A_31*x_4**6 + A_32*x_4**5 + A_33*x_4**4 + A_34*x_4**3 + A_35*x_4**2 + A_36*x**4 + A_37)),
             ('x_6', Eq(x_6, phi_a)),
             ('AA_1', Eq(AA_1, 

In [28]:
equations_ = [equation for equation in equations._asdict().values()]

sp.solve(equations_,
         symbols.B_n_BK,
         symbols.A_BK,
)

{A_BK: f_1*f_2*f_3, B_n_BK: exp*f_1*f_2*f_3*x_5*(B_B2K*x_3**B_B3K + B_BK)}